# Import module

In [1]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDataset

import os
import sys

c:\Users\hungu\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [39]:
# nếu có GPU thì sử dụng GPU để tăng tốc độ xử lý
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [40]:
# Load tokenizer và model đã train sẵn cho tiếng Việt từ NlpHUST
model_pretrained = 'NlpHUST/gpt2-vietnamese'
tokenizer = GPT2Tokenizer.from_pretrained(model_pretrained)
model = GPT2LMHeadModel.from_pretrained(model_pretrained)

# Load data

In [2]:
DATA_ROOT = '.'
filename = 'data.txt'
# Đọc dữ liệt từ file data.txt
with open(os.path.join(DATA_ROOT, filename), 'r', encoding='utf-8') as f:
  data = f.read()

In [ ]:
# chúng ta hoàn toàn có thể xử lý lọc dữ liệu ở đây (nhưng trong phần này mình không xử lý mà bỏ tất cả vào trong model cho nó tự học)

In [42]:
# max sequence length
MAX_LENGTH = 128
# dùng để làm bước nhảy để tạo sequence (thường là 1 nhưng để giảm số seq lại để train được nhiều dữ liệu mà vẫn giữ được độ chính xác nên mình sử dụng 32)
WINDOW_SIZE = 32

In [68]:
# giới hạn mức data lại để train model (do sức mạnh phần cứng không đáp ứng nổi)
limit_data = 700000 # 7 trăm nghìn ký tự
data_tokenized = tokenizer.encode(data[:limit_data]) 
len(data_tokenized), data_tokenized[:3]

(177642, [3643, 1361, 203])

In [69]:
def create_sequence(data, max_length=MAX_LENGTH, window_size=WINDOW_SIZE):
  # create seq data
  sequences = []
  mask = []
  for i in range(0, len(data)-max_length, window_size):
    sequences.append(data[i: i+max_length])
    mask.append([1 for _ in range(max_length)])
  return sequences, mask
sequences, masks = create_sequence(data_tokenized)
len(sequences)

5548

In [70]:
# Tạo data loader với batch size là 16
batch_size = 16

seq_inputs = torch.tensor(sequences)
seq_masks = torch.tensor(masks)

seq_data = TensorDataset(seq_inputs, seq_masks)
dataloader = DataLoader(seq_data, batch_size=batch_size) 

In [62]:
import time
import datetime

# Nhận đầu vào là giây, đầu ra là chuỗi hàng hh:mm:ss
def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

# Dùng để in ra tiến trình xử lý
def print_progress(index, total):
    sys.stdout.write('\r')
    progress = round(index*100/total)
    total_bar = 20
    sys.stdout.write('['+ '='*int(total_bar*progress/100) + '-'*int(total_bar-total_bar*progress/100) +'] ')
    sys.stdout.write(str(progress)+'%')
    sys.stdout.flush()

In [63]:
# hàm dùng để train model
def train(dataloader, model, optim, epochs):
  model.to(device)
  model.train()
  for i in range(epochs):
    print('======== Epoch {:} / {:} ========'.format(i + 1, epochs))
    print('Training...')
    t0 = time.time()
    total_loss = 0
    
    for step, batch in enumerate(dataloader):
      print_progress(step, len(dataloader))

      X = batch[0].to(device)
      m = batch[1].to(device)

      optim.zero_grad()

      loss = model(X, attention_mask=m, labels=X).loss
      total_loss += loss.item()

      loss.backward()
      optim.step()
    print("")
    print("  Average training loss: {0:.2f}".format(total_loss / len(dataloader)  ))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
    # Luu model nếu cần thiết
    torch.save(model.state_dict(), "model_state.pt")
    print('----------------------------------------------------------')
    print(infer("Tiếng Việt, còn gọi tiếng Việt Nam hay Việt ngữ, là"))
    print('----------------------------------------------------------')

# hàm dùng để in ra thực nghiệm
def infer(inp, max_generate_length=96):
  inp = tokenizer(inp, return_tensors="pt")
  X = inp["input_ids"].to(device)
  m = inp["attention_mask"].to(device)
  output = model.generate(X, attention_mask=m, max_length=max_generate_length)
  output = tokenizer.decode(output[0])
  return output

# Train model

In [71]:
# resize tầng embedding của model
model.resize_token_embeddings(len(tokenizer))

Embedding(50257, 768)

In [72]:
# khởi tại tuật toán tối ưu rồi train model
optim = Adam(model.parameters(), lr=1e-3)
train(dataloader, model, optim, epochs=12)

======== Epoch 1 / 12 ========
Training...
[====================] 100%
  Average training loss: 2.75
  Training epcoh took: 0:03:13


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


----------------------------------------------------------
Tiếng Việt, còn gọi tiếng Việt Nam hay Việt ngữ, là một ngôn ngữ thiểu số tại Việt Nam.

== Xem thêm ==
- "VietnameseVietnamese"
- "VietnameseVietnamese"
- "VietnameseVietnamese"
- "VietnameseVietnamese"
- "VietnameseVietnamese"
- "VietnameseVietnamese"
- "VietnameseVietnamese"
- "VietnameseVietnamese"
- "VietnameseVietnamese"
- "VietnameseVietnamese"
- "VietnameseVietnamese"
-
----------------------------------------------------------
======== Epoch 2 / 12 ========
Training...
[====================] 100%
  Average training loss: 1.89
  Training epcoh took: 0:03:13


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


----------------------------------------------------------
Tiếng Việt, còn gọi tiếng Việt Nam hay Việt ngữ, là ngôn ngữ của người Việt và là ngôn ngữ chính thức tại Việt Nam. Tiếng Việt còn là ngôn ngữ thứ hai của các dân tộc thiểu số tại Việt Nam.

== Xem thêm ==
- Học sinh Việt Nam
- Học sinh Việt Nam tại Mỹ
- Học sinh Việt Nam tại Canada
- Học sinh Việt Nam tại México
- Học sinh Việt Nam tại México
- Học sinh Việt Nam
----------------------------------------------------------
======== Epoch 3 / 12 ========
Training...
[====================] 100%
  Average training loss: 1.23
  Training epcoh took: 0:03:13


...

----------------------------------------------------------
Tiếng Việt, còn gọi tiếng Việt Nam hay Việt ngữ, là ngôn ngữ nhóm địa phương. Tiếng Việt còn là ngôn ngữ thứ hai của các dân tộc Tày, Nùng, Dao, Việt, Mường...

Cũng theo tiếng Việt, trong tiếng Việt còn có "Việt ngữ", có nghĩa là "Tiếng nói của người Việt ở hải ngoại".

== Tiếng địa phương ==
Tiếng Việt là ngôn ngữ có nguồn gốc bản địa, xuất thân từ nền văn minh nông
----------------------------------------------------------
======== Epoch 11 / 12 ========
Training...
[====================] 100%
  Average training loss: 0.08
  Training epcoh took: 0:03:12


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


----------------------------------------------------------
Tiếng Việt, còn gọi tiếng Việt Nam hay Việt ngữ, là các ngôn ngữ thường được dạy trong trường học và tại nhà do người Việt dạy, và được xếp vào Ngữ chi Việt thuộc nhóm ngôn ngữ Môn–Khmer trong hệ Nam Á ở khu vực Đông Nam Á hiện nay.

Trước năm 1954, tiếng Việt có một số từ vựng mượn từ tiếng Hán để biểu hiện sự phổ biến của tiếng Việt trong tiếng dân tộc thiểu số. Sau khi Việt Nam thống
----------------------------------------------------------
======== Epoch 12 / 12 ========
Training...
[====================] 100%
  Average training loss: 0.07
  Training epcoh took: 0:03:12


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


----------------------------------------------------------
Tiếng Việt, còn gọi tiếng Việt Nam hay Việt ngữ, là các ngôn ngữ chính thức tại Việt Nam. Tiếng Việt bao gồm cách phát âm tiếng Việt và chữ Quốc ngữ để viết. Tuy nhiên, hiện chưa có bất kỳ văn bản nào ở cấp nhà nước quy định "giọng chuẩn" và "quốc tự" của tiếng Việt. Hiện nay phần lớn các văn bản trong nước được viết theo những "Quy định về chính tả tiếng Việt và về thuật ngữ tiếng Việt"
----------------------------------------------------------


In [83]:
# chạy thực nghiệm model
print("infer from model : ")
while True:
  inp = input()
  if inp == 'exit':
    break
  print(infer(inp, 256))
  print()

infer from model : 
== Ngữ âm == ===Nguyên âm=== Giống như nhiều ngôn ngữ khác ở Đông Nam Á


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


== Ngữ âm == ===Nguyên âm=== Giống như nhiều ngôn ngữ khác ở Đông Nam Á, tiếng Việt khá phong phú về các nguyên âm. Dưới đây là bảng các nguyên âm theo giọng Hà Nội.
Trong bảng trên, các nguyên âm trước, giữa và nguyên âm mở là nguyên âm không tròn môi, còn lại là nguyên âm tròn môi. "Ă" và "â" là dạng ngắn của "a" và "ơ".

Đồng thời, tiếng Việt còn có hệ thống nguyên âm đôi và nguyên âm ba.

===Phụ âm===
Bảng dưới đây trình bày các phụ âm trong tiếng Việt và cách viết.
Một số phụ âm chỉ có một cách viết, nhưng một số có nhiều cách viết như "k", có thể được biểu diễn bằng "c", "k", "k", "k" hay "q".
Đồng thời, các phụ âm có nhiều cách viết như "k" và "qvist" có thể được biểu diễn bằng "c" hay "qvist nếu tất cả các người nói tiếng Việt đều đã nói tiếng Việt rành tiếng Anh ở nhà.

Tiếng Việt gọi là "phát thanh" hay "truyền hình" nhưng tiếng Anh gọi là "phát thanh" hay "truyền hình

Một số từ ngữ Hán cổ có thể kể đến như


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Một số từ ngữ Hán cổ có thể kể đến như "đầu", "gan", "ghế", "ông", "bà", "cô", "chè", "ngà", "chén", "chém", "chìm", "buồm", "mùi", "mùa hè mát mẻ" v.v. thì miền Bắc lại dùng những chữ "mùa đông", "tháng 6", "tháng 7" và "tháng 8". Vào mùa đông, các chữ cái bắt đầu có những dấu thanh, như "sỉ diện", "phi công", "bao gồm", "sống động", "sinh đẻ", vân vân. Trong khi tiếng Việt gọi là "phát thanh" thì tiếng Hán lại gọi là 廣播 "quảng bá"; tiếng Việt gọi là 電 電 電 電 電 電 電 電 電 電 電 電 電 電 電 電 電 電 電 電 電 電 電 電 電 電 電 電 電 電 電 電 電 電 電 電 電 電 電 電 電 電 電 電 電 電 電 電 電 電 電 電 �

Internet Society hay ISOC là


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Internet Society hay ISOC là một tổ chức quốc tế hoạt động vì lợi ích của cả cá nhân và cộng đồng. Tổ chức này chú trọng đến: tiêu chuẩn, giáo dục và các vấn đề về chính sách. Với trên 145 tổ chức thành viên và 65.000 thành viên cá nhân, ISOC bao gồm những con người cụ thể trong cộng đồng Internet. Mọi chi tiết có thể tìm thấy tại website của ISOC.

Internet Society nằm ở gần thủ đô Washington, DC, Hoa Kỳ và Geneva, Thụy Sĩ. Số hội viên của nó bao gồm hơn 145 tổ chức thành viên đến từ khắp nơi trên khắp quốc gia, hơn 65.000 người ở Thụy Điển, Úc, Hàn Quốc, Israel, Hàn Quốc, và gần 40.000 thành viên thuộc các Cộng hòa Xã hội Chủ nghĩa Việt Nam. Thành viên còn có thể tự lập một chi nhánh của mình.

== Các cựu thuộc địa của Liên bang ==
Theo nguyên tắc căn bản, tất cả các cựu thuộc địa của Liên bang trước khi trở thành thuộc địa của Hoa Kỳ phải được tái bầu cử lại. Các ứng cử viên Cộng hòa đương nhiệm thuộc Đảng Cộng hòa đương nhiệm thuộc Đảng Cộng hòa đương nhiệm thường lệ kiểm soát cả h

### Trong dữ liệu huấn luyện
1.
== Ngữ âm ==
===Nguyên âm===
Giống như nhiều ngôn ngữ khác ở Đông Nam Á, tiếng Việt khá phong phú về nguyên âm. Dưới đây là bảng các nguyên âm theo giọng Hà Nội.
Trong bảng trên, các nguyên âm trước, giữa và nguyên âm mở là nguyên âm không tròn môi, còn lại là nguyên âm tròn môi. "Ă" và "â" là dạng ngắn của "a" và "ơ".

Đồng thời, tiếng Việt còn có hệ thống nguyên âm đôi và nguyên âm ba.

2. 
Một số từ ngữ Hán cổ có thể kể đến như "đầu", "gan", "ghế", "ông", "bà", "cô", "chè", "ngà", "chén", "chém", "chìm", "buồng", "buồn", "buồm", "mùi", "mùa"... Từ Hán cổ là những từ gốc Hán được du nhập vào tiếng Việt đã lâu, đã được đồng hoá rất mạnh, nên những từ này hiện nay là từ thông thường trong hoạt động xã hội đối với người Việt.
Hệ thống Hán–Việt trong tiếng Việt bằng cách đọc các chữ Hán theo ngữ âm hiện có của tiếng Việt. Số lượng từ vựng tiếng Việt có thêm hàng loạt các yếu tố Hán–Việt. Như là "chủ", "ở", "tâm", "minh", "đức", "thiên", "tự do"... giữ nguyên nghĩa chỉ khác cách đọc; hay thay đổi vị trí như "nhiệt náo" thành "náo nhiệt", "thích phóng" thành "phóng thích", "đảm bảo" thành "bảo đảm"...; hoặc được rút gọn như "thừa trần" thành "trần", "lạc hoa sinh" thành "lạc"...; hoặc được đọc chệch đi như "tiếp thu" thành "tiếp thụ", "tháp nhập" thành "sáp nhập", "thống kế" thành "thống kê", "chúng cư" thành "chung cư", "vãn cảnh" thành "vãng cảnh", "khuyến mãi" thành "khuyến mại"...; 

3. 
Internet Society

Internet Society hay ISOC là một tổ chức quốc tế hoạt động phi lợi nhuận, phi chính phủ và bao gồm các thành viên có trình độ chuyên ngành. Tổ chức này chú trọng đến: tiêu chuẩn, giáo dục và các vấn đề về chính sách. Với trên 145 tổ chức thành viên và 65.000 thành viên cá nhân, ISOC bao gồm những con người cụ thể trong cộng đồng Internet. Mọi chi tiết có thể tìm thấy tại website của ISOC.

Internet Society nằm ở gần thủ đô Washington, DC, Hoa Kỳ và Geneva, Thụy Sĩ. Số hội viên của nó bao gồm hơn 145 tổ chức thành viên và hơn 65.000 cá nhân. Thành viên còn có thể tự lập một chi nhánh của tổ chức tùy theo vị trí hoặc sở thích. Hiện nay tổ chức có tới 90 chi nhánh trên toàn thế giới.
